# Logistic Regression

A client wants to reduce customer churn by determining how reliably it can be predicted and intervening in situations where a client is likely to leave them. We'll use the data in customer_churn.csv to fit and evaluate a model and then generate predictions using the data in new_customers.csv.

In [1]:
import findspark
findspark.init("/home/bryan/Documents/Code/spark-2.4.5-bin-hadoop2.7")

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_churn').getOrCreate()

# EDA

In [3]:
data = spark.read.csv("data/customer_churn.csv", inferSchema=True, header=True)

In [4]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [5]:
assert data.count() == data.na.drop().count(), "Check for missing data."

In [6]:
data.show(3)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 3 rows



> ### According to the client, whether a client has an account manager or not is determined randomly. Can we see, based on the data, whether this random phenomenon has a positive or negative impact on churn?

In [7]:
for i in ['Age','Total_Purchase','Account_Manager','Years','Num_Sites']:
    this_corr = data.stat.corr(i, 'Churn')
    print(i, round(this_corr,3))

Age 0.086
Total_Purchase 0.024
Account_Manager 0.071
Years 0.214
Num_Sites 0.525


> ### Checking Pearson's correlation between several numerical features and the label (Churn) indicates that Num_Sites and Years have a positive, if weak, correlation while the others have significantly weaker positive correlations.

> ### Surprisingly, Total_Purchase has almost the smallest correlation with Churn.

In [8]:
round(data.select('Company').distinct().count()/data.count(),2)

0.97

> ### The Company column is effectively a unique value and may not help us in predicting churn.

In [9]:
data.select('Location').show(10, False)

+-------------------------------------------------------+
|Location                                               |
+-------------------------------------------------------+
|10265 Elizabeth Mission Barkerburgh, AK 89518          |
|6157 Frank Gardens Suite 019 Carloshaven, RI 17756     |
|1331 Keith Court Alyssahaven, DE 90114                 |
|13120 Daniel Mount Angelabury, WY 30645-4695           |
|765 Tricia Row Karenshire, MH 71730                    |
|6187 Olson Mountains East Vincentborough, PR 74359     |
|4846 Savannah Road West Justin, IA 87713-3460          |
|25271 Roy Expressway Suite 147 Brownport, FM 59852-6150|
|3725 Caroline Stravenue South Christineview, MA 82059  |
|363 Sandra Lodge Suite 144 South Ann, WI 51655-7561    |
+-------------------------------------------------------+
only showing top 10 rows



> ### The locations appear to be U.S.-based. Would it be helpful to generate new State and/or Zip Code features? We will try that below.

In [10]:
from pyspark.sql import functions as F

In [11]:
data = data.withColumn('Onboard_date_year', F.year('Onboard_date'))
data.select('Onboard_date_year').describe().show()

+-------+------------------+
|summary| Onboard_date_year|
+-------+------------------+
|  count|               900|
|   mean|2010.8011111111111|
| stddev|3.2072288498508783|
|    min|              2006|
|    max|              2016|
+-------+------------------+



> ### Onboard_date covers a period of approximately 10 years from 2006 to 2016. Below we will verify whether the columns Years and Onboard_date are reliable.

In [12]:
data = data.withColumn('Years_check', F.col('Onboard_date_year')+F.col('Years'))
data.select('Years_check').describe().show()

+-------+------------------+
|summary|       Years_check|
+-------+------------------+
|  count|               900|
|   mean|2016.0742666666667|
| stddev|3.3954957103523697|
|    min|           2008.41|
|    max|            2023.6|
+-------+------------------+



In [13]:
data.filter(F.col('Years_check')>2021).select(['Years', 'Onboard_date_year', 'Years_check']).show(5)

+-----+-----------------+-----------+
|Years|Onboard_date_year|Years_check|
+-----+-----------------+-----------+
| 6.67|             2016|    2022.67|
| 5.56|             2016|    2021.56|
| 5.23|             2016|    2021.23|
| 5.22|             2016|    2021.22|
| 6.64|             2015|    2021.64|
+-----+-----------------+-----------+
only showing top 5 rows



In [14]:
data.filter(F.col('Years_check')>2021).count()

60

> ### At least 60 records have a combination of 'Years' and 'Onboard_date' that indicates the client has been with the company beyond the year 2020. This does not make sense. However, we'll simply use the 'Years' column in the interest of this toy exercise.

# Feature Extraction: State & Zip Code

In [15]:
this_pattern = r"([A-Z]{2}) (\d{5})"
data = data.withColumn('Location_state', F.regexp_extract(F.col('Location'),this_pattern,1))\
.withColumn('Location_zip', F.regexp_extract(F.col('Location'),this_pattern,2))

In [16]:
data.select(['Location', 'Location_state', 'Location_zip']).show(5, False)

+--------------------------------------------------+--------------+------------+
|Location                                          |Location_state|Location_zip|
+--------------------------------------------------+--------------+------------+
|10265 Elizabeth Mission Barkerburgh, AK 89518     |AK            |89518       |
|6157 Frank Gardens Suite 019 Carloshaven, RI 17756|RI            |17756       |
|1331 Keith Court Alyssahaven, DE 90114            |DE            |90114       |
|13120 Daniel Mount Angelabury, WY 30645-4695      |WY            |30645       |
|765 Tricia Row Karenshire, MH 71730               |MH            |71730       |
+--------------------------------------------------+--------------+------------+
only showing top 5 rows



In [17]:
data.groupby('Location_state').count().orderBy('count', ascending=False).show(10)

+--------------+-----+
|Location_state|count|
+--------------+-----+
|            AA|   44|
|            AP|   36|
|            AE|   27|
|            OK|   22|
|            SC|   18|
|            WV|   18|
|            TN|   18|
|            ME|   17|
|            MN|   17|
|            AL|   17|
+--------------+-----+
only showing top 10 rows



In [18]:
data.groupby('Location_zip').count().orderBy('count', ascending=False).show(10)

+------------+-----+
|Location_zip|count|
+------------+-----+
|       19037|    2|
|       12062|    2|
|       92778|    2|
|       26077|    2|
|       68644|    2|
|       62096|    2|
|       96632|    1|
|       24477|    1|
|       81808|    1|
|       67338|    1|
+------------+-----+
only showing top 10 rows



> ### It appears the state attribute from 'Location' may be useful for our model. The zip code attribute is likely too unique to help fit a predictive model.

# Baseline model

In [19]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

In [20]:
state_indexer = StringIndexer(inputCol='Location_state', outputCol='StateIndex')
state_encoder = OneHotEncoder(inputCol='StateIndex',outputCol='StateVec')

In [21]:
assembler = VectorAssembler(inputCols=[
    'Age',
    'Total_Purchase',
    'Account_Manager',
    'Years',
    'Num_Sites'
], outputCol='features')

In [22]:
from pyspark.ml.classification import LogisticRegression

In [23]:
from pyspark.ml import Pipeline

In [24]:
log_reg_base = LogisticRegression(featuresCol='features',labelCol='Churn')

In [25]:
train_data, test_data = data.randomSplit([0.7,.3])

## Create a pipeline for pre-processing, assembling the feature vector, & fitting a model

In [26]:
pipeline = Pipeline(stages=[state_indexer,state_encoder,
                           assembler,log_reg_base])

log_reg_base_fit = pipeline.fit(train_data)
log_reg_base_results = log_reg_base_fit.transform(train_data)

In [27]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [35]:
base_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                          labelCol='Churn')

base_AUC = base_eval.evaluate(log_reg_base_results)
base_AUC

0.8010244355956287

# Better model

In [45]:
assembler = VectorAssembler(inputCols=[
    'Age',
    'Total_Purchase',
    'Account_Manager',
    'Years',
    'Num_Sites',
    'StateVec'
], outputCol='features')

In [46]:
pipeline = Pipeline(stages=[state_indexer,state_encoder,
                           assembler,log_reg_base])

better_model = pipeline.fit(train_data)
better_model_results = better_model.transform(train_data)

In [47]:
better_AUC = base_eval.evaluate(better_model_results)
better_AUC

0.8524391741365095

In [66]:
round((better_AUC-base_AUC)*100,3)

5.141

> ### Adding a feature related to the U.S. state improved performance by >5%. Next we will evaluate the impact of regularization on the model.

In [56]:
log_reg_l1 = LogisticRegression(featuresCol='features',labelCol='Churn', regParam=0.0, elasticNetParam=0.0)
raise ValueError('How to properly set l1, l2, elasticnet regularization with the above params?')

ValueError: How to properly set l1, l2, elasticnet regularization with the above params?

# End